In [ ]:
# https://www.kaggle.com/pmigdal/alien-vs-predator-images

In [ ]:
from tensorflow.keras.applications import Xception
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception

In [ ]:
input_shape = (150, 150, 3)
target_size = (150, 150)

batch_size = 16

In [ ]:
def visualize_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

In [ ]:
def build_base_cnn_model():
    model = Sequential()
    model.add(Conv2D(input_shape=input_shape, filters=64, kernel_size=(4,4), strides=(2)))
    model.add(Activation('relu'))
    model.add(Conv2D(filters=64, kernel_size=(4,4), strides=(1)))
    model.add(MaxPooling2D(pool_size=4))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer = 'adam', 
        loss = 'binary_crossentropy', 
        metrics = ['accuracy']
    )
    return model

base_cnn_model = build_base_cnn_model()
base_cnn_model.summary()

In [ ]:
train_samples = 694
test_samples = 200

training_steps = train_samples // batch_size
validation_steps = test_samples // batch_size


def get_train_test_generator():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
    )

    test_datagen = ImageDataGenerator(
        rescale=1./255,
    )

    train_generator = train_datagen.flow_from_directory(
        "data/train",
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary'
    )

    validation_generator = test_datagen.flow_from_directory(
        "data/validation",
        batch_size=batch_size,
        target_size=target_size,
        class_mode='binary'
    )
    return train_generator, validation_generator

In [ ]:
history_base_cnn = base_cnn_model.fit_generator(
    train_generator,  
    steps_per_epoch=train_samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=test_samples // batch_size
)

base_cnn_model.save("base_cnn_model.h5")
visualize_history(history_base_cnn)

# Data Augmentation

In [ ]:
def get_augmented_train_test_generator():
        
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=10,
        zoom_range=0.2,
        horizontal_flip=True,
    )

    test_datagen = ImageDataGenerator(
        rescale=1./255,
    )

    train_generator = train_datagen.flow_from_directory(
        "data/train",
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary'
    )

    validation_generator = test_datagen.flow_from_directory(
        "data/validation",
        batch_size=batch_size,
        target_size=target_size,
        class_mode='binary'
    )
    return train_generator, validation_generator


train_generator, validation_generator = get_augmented_train_test_generator()
base_cnn_model_with_augmentation = build_base_cnn_model()

history_base_augmented_cnn = base_cnn_model_with_augmentation.fit_generator(
    train_generator,
    steps_per_epoch=training_steps,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps
)


base_cnn_model_with_augmentation.save("base_cnn_model_with_augmentation.h5")
visualize_history(history_base_augmented_cnn)

# Transfer Learning

In [ ]:
def create_xception_model():
    conv_base = Xception(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )

    for layer in conv_base.layers:
        layer.trainable = False

    xception = models.Sequential()
    xception.add(conv_base)
    xception.add(layers.GlobalAveragePooling2D())
    xception.add(layers.Dense(1, activation='softmax'))
    xception.compile(
        optimizer = 'adam', 
        loss = 'binary_crossentropy', 
        metrics = ['accuracy']
    )
    return xception


xception = create_xception_model()
train_generator, validation_generator = get_augmented_train_test_generator()

xception_history = xception.fit_generator(
    generator=train_generator,
    steps_per_epoch=347 // 32,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=10
)

xception.save("xception.h5")
visualize_history(history_base_augmented_cnn)

## Fine Tuning

In [ ]:
conv_base = Xception(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape
)

for layer in conv_base.layers[:-5]:
    layer.trainable = False

xception_fine_tuning = create_xception_model()

xception_history = xception_fine_tuning.fit_generator(
    generator=train_generator,
    steps_per_epoch=347 // 32,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=10
)

xception_fine_tuning.save("xception_fine_tuning.h5")